In [16]:
import pandas as pd
# Read in data
df = pd.read_csv('./data/train.csv')
df.columns = map(str.lower, df.columns)
df.head()

,loan_id,gender,married,dependents,education,self_employed,applicantincome,coapplicantincome,loanamount,loan_amount_term,credit_history,property_area,loan_status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [17]:
# Fix value from int to float
df["applicantincome"] = df["applicantincome"].astype("float64")

In [18]:
# show null values
df.isnull().sum().sort_values(ascending=False)

credit_history       50
self_employed        32
loanamount           22
dependents           15
loan_amount_term     14
gender               13
married               3
loan_status           0
property_area         0
coapplicantincome     0
applicantincome       0
education             0
loan_id               0
dtype: int64

# Data Deletion

In [19]:
# Drop some features
df.drop(["loan_id", "gender"], axis=1, inplace=True)

# Data Preparation

### Label Encoding

In [31]:
## Label Encoding
# Remove label
# Target is "loan_status"
from sklearn.preprocessing import LabelEncoder  
le = LabelEncoder()
# transform the target column

target_values = {'Y': 0 , 'N' : 1}

target = df['loan_status']
features = df.drop('loan_status', axis=1)

target = target.map(target_values)

### Numeric and categorical features

In [32]:
## Numeric and categorical features
from pandas.api.types import is_numeric_dtype

categorical_features = []
numerical_features = []

for column in features.columns:
    if is_numeric_dtype(features[column]):
        numerical_features.append(column)
    else :
        categorical_features.append(column)
        
print(f"Numerical features: {numerical_features}")
print(f"Categorical features: {categorical_features}")

Numerical features: ['applicantincome', 'coapplicantincome', 'loanamount', 'loan_amount_term', 'credit_history']
Categorical features: ['married', 'dependents', 'education', 'self_employed', 'property_area']


### Missing data

In [52]:
numerical_data = features[numerical_features]
numerical_data = numerical_data.apply(lambda x:x.fillna(x.mean()))
categorical_data = features[categorical_features]
categorical_data = categorical_data.apply(lambda x:x.fillna("-"))

### Data Splitting

In [59]:
X = pd.concat([numerical_data, categorical_data], axis=1)
y = target 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

### Numerical features: Missing data and MinMax Scaling

In [37]:
# MinMax Scaling
numerical_data_std = (numerical_data - numerical_data.min(axis=0)) / (numerical_data.max(axis=0) - numerical_data.min(axis=0))
numerical_scaled = numerical_data_std * (1 - 0) + 1
numerical_scaled.isnull().sum().any() # no more missing data 

False

### Categorical features: Feature Engineering with One-Hot-Encoding

In [43]:
for feature in categorical_features:
    one_hot = pd.get_dummies(categorical_data[feature])
    for column in one_hot.columns:
        categorical_data[f"{feature}_{column}"] = one_hot[column]

categorical_data.drop(categorical_features, axis=1, inplace=True)

In [44]:
categorical_data.head()

,married_-,married_No,married_Yes,dependents_-,dependents_0,dependents_1,dependents_2,dependents_3+,education_Graduate,education_Not Graduate,self_employed_-,self_employed_No,self_employed_Yes,property_area_Rural,property_area_Semiurban,property_area_Urban
0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1
1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0
2,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1
3,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1
4,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1


# Train Model

In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [47]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

In [48]:
X.head()

,applicantincome,coapplicantincome,loanamount,loan_amount_term,credit_history,married_-,married_No,married_Yes,dependents_-,dependents_0,...,dependents_2,dependents_3+,education_Graduate,education_Not Graduate,self_employed_-,self_employed_No,self_employed_Yes,property_area_Rural,property_area_Semiurban,property_area_Urban
0,1.070489,1.000000,1.198860,1.74359,2.0,0,1,0,0,1,...,0,0,1,0,0,1,0,0,0,1
1,1.054830,1.036192,1.172214,1.74359,2.0,0,0,1,0,0,...,0,0,1,0,0,1,0,1,0,0
2,1.035250,1.000000,1.082489,1.74359,2.0,0,0,1,0,1,...,0,0,1,0,0,0,1,0,0,1
3,1.030093,1.056592,1.160637,1.74359,2.0,0,0,1,0,1,...,0,0,0,1,0,1,0,0,0,1
4,1.072356,1.000000,1.191027,1.74359,2.0,0,1,0,0,1,...,0,0,1,0,0,1,0,0,0,1


In [49]:
clf.fit(X_train, y_train)

LogisticRegression()

In [50]:
from sklearn.metrics import precision_score, recall_score, f1_score, log_loss, accuracy_score

def loss(y_true, y_pred, retu=False):
    pre = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    loss = log_loss(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    
    if retu:
        return pre, rec, f1, loss, acc
    else:
        print('  pre: %.3f\n  rec: %.3f\n  f1: %.3f\n  loss: %.3f\n  acc: %.3f' % (pre, rec, f1, loss, acc))
        
loss(clf.predict(X_test), y_test)    

  pre: 0.440
  rec: 0.898
  f1: 0.591
  loss: 6.863
  acc: 0.801


# Pipeline

In [66]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, numerical_features),
        ('categorical', categorical_transformer, categorical_features)])

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', clf)])

In [67]:
model = pipeline.fit(X_train,y_train)

In [68]:
loss(model.predict(X_test), y_test)

  pre: 0.440
  rec: 0.898
  f1: 0.591
  loss: 6.863
  acc: 0.801


# skl2onnx

In [84]:
#!pip install skl2onnx
#!pip install onnxruntime

### Input preparation

In [73]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType, StringTensorType, Int64TensorType

def convert_dataframe_schema(df, drop=None):
    inputs = []
    for k, v in zip(df.columns, df.dtypes):
        if drop is not None and k in drop:
            continue
        #if v == 'int64':
        #    t = Int64TensorType([None, 1])
        if v == 'float64':
            t = FloatTensorType([None, 1])
        else:
            t = StringTensorType([None, 1])
        inputs.append((k, t))
    return inputs

inputs = []
for categorical in categorical_features:
    inputs.append((categorical, StringTensorType([None, 1])))
for numerical in numerical_features:
    inputs.append((numerical, FloatTensorType([None, 1])))
#inputs = convert_dataframe_schema(X)


In [74]:
inputs

[('married', StringTensorType(shape=[None, 1])),
 ('dependents', StringTensorType(shape=[None, 1])),
 ('education', StringTensorType(shape=[None, 1])),
 ('self_employed', StringTensorType(shape=[None, 1])),
 ('property_area', StringTensorType(shape=[None, 1])),
 ('applicantincome', FloatTensorType(shape=[None, 1])),
 ('coapplicantincome', FloatTensorType(shape=[None, 1])),
 ('loanamount', FloatTensorType(shape=[None, 1])),
 ('loan_amount_term', FloatTensorType(shape=[None, 1])),
 ('credit_history', FloatTensorType(shape=[None, 1]))]

In [89]:
# Write model
odel_onnx = convert_sklearn(pipeline, initial_types=inputs)
with open("model/loan_model.onnx", "wb") as f:
    f.write(model_onnx.SerializeToString())

In [79]:
import numpy as np

In [80]:
# Prepare input data
input_data = {column: X_test[column].values for column in X_test.columns}
for numeric in numerical_features:
    input_data[numeric] = input_data[numeric].astype(np.float32)
for categorical in input_data:
    input_data[categorical] = input_data[categorical].reshape((input_data[categorical].shape[0], 1))

In [90]:
import onnxruntime as rt
sess = rt.InferenceSession("model/loan_model.onnx")
pred_onnx = sess.run(None, input_data)

In [91]:
loss(pred_onnx[0], model.predict(X_test))

  pre: 1.000
  rec: 1.000
  f1: 1.000
  loss: 0.000
  acc: 1.000
